# 00 - Spotify API


In [21]:
# import libraries
from dotenv import load_dotenv
import os
import pandas as pd
import requests
import time
from tqdm.auto import tqdm

# Load environment variables from .env file
load_dotenv()

# Access the API key
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [22]:
# request access token
url = "https://accounts.spotify.com/api/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
}
headers = {"Content-Type": "application/x-www-form-urlencoded"}
response = requests.post(url, data=payload, headers=headers)
response.raise_for_status()  # Raise an exception for HTTP errors
access_token = response.json()["access_token"]


In [23]:
# sample playlist dataframe
playlists = pd.read_csv("./data/playlists.csv")
playlists

,playlist_url,playlist_name
0,https://open.spotify.com/playlist/37i9dQZF1DXb...,Lofi House
1,https://open.spotify.com/playlist/37i9dQZF1DX1...,hard Rock
2,https://open.spotify.com/playlist/37i9dQZF1DX9...,West Coast Classics
3,https://open.spotify.com/playlist/37i9dQZF1DWT...,Summer Jazz


## Track meta


In [24]:
def get_track_id_from_url(track_url):
    """Extract track ID from Spotify URL."""
    return track_url.split("/")[-1].split("?")[0]


# function to get hit Spotfy API and get playlist metadata
def get_tracks_metadata(track_inputs, access_token=access_token):
    """
    Fetch metadata for Spotify tracks.

    Parameters:
    - track_inputs: A list or string of Spotify track URLs or IDs.
    - access_token: Your Spotify API Bearer token

    Returns:
    - A dataframe containing the tracks metadata
    """
    if isinstance(track_inputs, str):
        track_inputs = [track_inputs]

    track_ids = [
        get_track_id_from_url(track) if "spotify.com" in track else track
        for track in track_inputs
    ]

    endpoint = "https://api.spotify.com/v1/tracks"
    headers = {"Authorization": f"Bearer {access_token}"}

    # Initialize a list to hold all tracks data
    all_tracks = []

    # Spotify API allows max 100 IDs per request, so we process in batches
    batch_size = 50

    # looping through the track_ids in batches
    for i in range(0, len(track_ids), batch_size):
        batch_ids = track_ids[i : i + batch_size]
        ids = ",".join(batch_ids)
        params = {"ids": ids}

        try:
            # Make a GET request to the audio features endpoint
            response = requests.get(endpoint, headers=headers, params=params)
            response.raise_for_status()  # Raise an exception for HTTP errors
            tracks_meta = response.json()
            all_tracks.extend(tracks_meta["tracks"])

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            continue

    # Convert the list of tracks to a DataFrame
    all_tracks = pd.DataFrame(all_tracks)

    # dropping unnecessary columns
    all_tracks = all_tracks[
        [
            "id",
            "name",
            "album",
            "artists",
            "available_markets",
            "disc_number",
            "duration_ms",
            "explicit",
            "external_ids",
            "external_urls",
            "is_local",
            "popularity",
            "preview_url",
            "track_number",
            "type",
            "uri",
        ]
    ].reset_index(drop=True)

    return all_tracks


In [25]:
# sample tracks
tracks_id = ["0mflMxspEfB0VbI1kyLiAv", "4tekXLedu6wxNhHkKjFkPE"]

# getting the meta
meta = get_tracks_metadata(tracks_id, access_token)
meta

,id,name,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,is_local,popularity,preview_url,track_number,type,uri
0,0mflMxspEfB0VbI1kyLiAv,Stick Season,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,182346,False,{'isrc': 'USUM72212470'},{'spotify': 'https://open.spotify.com/track/0m...,False,88,NaN,2,track,spotify:track:0mflMxspEfB0VbI1kyLiAv
1,4tekXLedu6wxNhHkKjFkPE,Let's Vibe,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,193663,False,{'isrc': 'FXQ892400160'},{'spotify': 'https://open.spotify.com/track/4t...,False,53,https://p.scdn.co/mp3-preview/4bc677efe55ee736...,1,track,spotify:track:4tekXLedu6wxNhHkKjFkPE


## Playlist meta


In [26]:
# function to get hit Spotfy API and get playlist metadata
def get_playlist_metadata(playlist_url, access_token=access_token):
    """
    Fetch metadata for a Spotify playlist.

    Parameters:
    - playlist_id: The Spotify playlist ID
    - access_token: Your Spotify API Bearer token

    Returns:
    - A dictionary containing the playlist metadata
    """

    # Extract playlist ID from URL if a full URL is provided
    playlist_id = playlist_url.split("/")[-1]
    if "?" in playlist_id:
        playlist_id = playlist_id.split("?")[0]

    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = {"Authorization": f"Bearer {access_token}"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()

        # Extract relevant metadata
        metadata = {
            "playlist_name": data["name"],
            "playlist_id": data["id"],
            "owner": data["owner"]["display_name"],
            "description": data["description"],
            "followers": data["followers"]["total"],
            "link": data["external_urls"]["spotify"],
            "image": data["images"][0]["url"],
        }

        return metadata

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

In [27]:
# sample playlist
playlist_url = "https://open.spotify.com/playlist/37i9dQZF1DXbXD9pMSZomS"

# getting the meta
meta = get_playlist_metadata(playlist_url, access_token)
meta

{'playlist_name': 'Lo-Fi House',
 'playlist_id': '37i9dQZF1DXbXD9pMSZomS',
 'owner': 'Spotify',
 'description': 'Distorted and rough around the edges.',
 'followers': 476129,
 'link': 'https://open.spotify.com/playlist/37i9dQZF1DXbXD9pMSZomS',
 'image': 'https://i.scdn.co/image/ab67706f00000002274b8704a985caf11325cc03'}

In [28]:
pd.DataFrame([meta])

,playlist_name,playlist_id,owner,description,followers,link,image
0,Lo-Fi House,37i9dQZF1DXbXD9pMSZomS,Spotify,Distorted and rough around the edges.,476129,https://open.spotify.com/playlist/37i9dQZF1DXb...,https://i.scdn.co/image/ab67706f00000002274b87...


## Playlist track List


In [29]:
def get_playlist_tracks(playlist_url, access_token):
    """
    Fetch all tracks from a Spotify playlist.

    Parameters:
    - playlist_id: The Spotify playlist ID
    - access_token: Your Spotify API Bearer token

    Returns:
    - A dataframe of all tracks in the playlist
    """

    # Extract playlist ID from URL if a full URL is provided
    playlist_id = playlist_url.split("/")[-1]
    if "?" in playlist_id:
        playlist_id = playlist_id.split("?")[0]

    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {"Authorization": f"Bearer {access_token}"}

    track_list = []
    params = {"limit": 50, "offset": 0}

    while True:
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()  # Raise an exception for HTTP errors
            data = response.json()

            # Extract the 'track' value from each item
            tracks_data = [
                track["track"] for track in data["items"] if track["track"] is not None
            ]

            # Extract track items from the response
            track_list.extend(tracks_data)

            # Check if there are more tracks to fetch
            if data["next"]:
                params["offset"] += params["limit"]
                time.sleep(1)  # To avoid hitting rate limits
            else:
                break

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break
        except KeyError as e:
            print(f"Unexpected response format: {e}")
            break

    # Create a DataFrame
    df = pd.DataFrame(track_list)

    # adding the playlist id to the dataframe
    df["playlist_url"] = playlist_url
    df["playlist_id"] = playlist_id

    return df

In [30]:
# sample playlist
playlist_url = "https://open.spotify.com/playlist/37i9dQZF1DX9ND1QF5hZNF"

# getting the tracks
tracks = get_playlist_tracks(playlist_url, access_token)

In [31]:
tracks.head()

,preview_url,available_markets,explicit,type,episode,track,album,artists,disc_number,track_number,...,external_ids,external_urls,href,id,name,popularity,uri,is_local,playlist_url,playlist_id
0,None,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'DECE72401347'},{'spotify': 'https://open.spotify.com/track/5n...,https://api.spotify.com/v1/tracks/5nPbKG04fhLk...,5nPbKG04fhLkIAjcPFaZq7,I Adore You (feat. Daecolm),84,spotify:track:5nPbKG04fhLkIAjcPFaZq7,False,https://open.spotify.com/playlist/37i9dQZF1DX9...,37i9dQZF1DX9ND1QF5hZNF
1,https://p.scdn.co/mp3-preview/436a259f68bc1717...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'DEH742427428'},{'spotify': 'https://open.spotify.com/track/7d...,https://api.spotify.com/v1/tracks/7d6bWTHr7EZg...,7d6bWTHr7EZgygynnxAKCA,Electric Feel,55,spotify:track:7d6bWTHr7EZgygynnxAKCA,False,https://open.spotify.com/playlist/37i9dQZF1DX9...,37i9dQZF1DX9ND1QF5hZNF
2,https://p.scdn.co/mp3-preview/cf318e72f0fc4472...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'SE5W32204551'},{'spotify': 'https://open.spotify.com/track/35...,https://api.spotify.com/v1/tracks/35FAJSbOqHs0...,35FAJSbOqHs0lo5U7Ty0o9,Blend,50,spotify:track:35FAJSbOqHs0lo5U7Ty0o9,False,https://open.spotify.com/playlist/37i9dQZF1DX9...,37i9dQZF1DX9ND1QF5hZNF
3,https://p.scdn.co/mp3-preview/dc3f5560d882d3f9...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'USA2P2439321'},{'spotify': 'https://open.spotify.com/track/4M...,https://api.spotify.com/v1/tracks/4MSj19TwYBLg...,4MSj19TwYBLgDFj3ddEeco,Black Friday (pretty like the sun),85,spotify:track:4MSj19TwYBLgDFj3ddEeco,False,https://open.spotify.com/playlist/37i9dQZF1DX9...,37i9dQZF1DX9ND1QF5hZNF
4,https://p.scdn.co/mp3-preview/bacd8fab60c68555...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'SE4RG2306229'},{'spotify': 'https://open.spotify.com/track/40...,https://api.spotify.com/v1/tracks/40rev08jSE8Z...,40rev08jSE8ZQfrESVu4uc,BWU,57,spotify:track:40rev08jSE8ZQfrESVu4uc,False,https://open.spotify.com/playlist/37i9dQZF1DX9...,37i9dQZF1DX9ND1QF5hZNF


## Audio features


In [32]:
def get_audio_features(track_ids, access_token):
    """
    Fetch audio features for a list of Spotify track IDs and return them as a DataFrame.

    Parameters:
    - track_ids: A list or array of Spotify track IDs.
    - access_token: Your Spotify API Bearer token.

    Returns:
    - A pandas DataFrame with track IDs as the index and audio features as columns.
    """
    endpoint = "https://api.spotify.com/v1/audio-features"
    headers = {"Authorization": f"Bearer {access_token}"}

    # Initialize a list to hold all audio feature data
    all_audio_features = []

    # Spotify API allows max 100 IDs per request, so we process in batches
    batch_size = 100
    if isinstance(track_ids, str):
        track_ids = [track_ids]

    # looping through the track_ids in batches
    for i in range(0, len(track_ids), batch_size):
        batch_ids = track_ids[i : i + batch_size]
        ids = ",".join(batch_ids)
        params = {"ids": ids}

        try:
            # Make a GET request to the audio features endpoint
            response = requests.get(endpoint, headers=headers, params=params)
            response.raise_for_status()  # Raise an exception for HTTP errors
            audio_features = response.json()["audio_features"]
            all_audio_features.extend(audio_features)

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            continue

    # Convert the list of audio features to a DataFrame
    audio_features_df = pd.DataFrame(all_audio_features)

    # dropping unnecessary columns
    audio_features_df.drop(
        ["type", "uri", "track_href", "analysis_url", "duration_ms"],
        axis=1,
        inplace=True,
    )

    return audio_features_df.reset_index()

In [33]:
# example of getting audio features
af = get_audio_features(
    ["4MSj19TwYBLgDFj3ddEeco", "1VKqsNsQvM0oCJKzUtWD2H"], access_token
)
af

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature
0,0,0.853,0.618,6,-10.833,1,0.0923,0.633,0.89000,0.0972,0.320,126.031,4MSj19TwYBLgDFj3ddEeco,4
1,1,0.642,0.734,11,-8.091,0,0.0485,0.135,0.00216,0.3110,0.686,120.029,1VKqsNsQvM0oCJKzUtWD2H,4


In [34]:
# getting audio features for all the tracks in the playlist
af_df = get_audio_features(tracks["id"].values[:4], access_token)
af_df

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature
0,0,0.704,0.787,11,-6.751,0,0.0462,0.009260,0.00538,0.1590,0.3100,119.996,5nPbKG04fhLkIAjcPFaZq7,4
1,1,0.528,0.909,5,-7.839,0,0.0343,0.000757,0.01530,0.1120,0.0772,119.990,7d6bWTHr7EZgygynnxAKCA,4
2,2,0.574,0.360,0,-16.604,1,0.0859,0.503000,0.92400,0.1050,0.1650,113.863,35FAJSbOqHs0lo5U7Ty0o9,4
3,3,0.853,0.618,6,-10.833,1,0.0923,0.633000,0.89000,0.0972,0.3200,126.031,4MSj19TwYBLgDFj3ddEeco,4


## Putting it together


In [35]:
# getting track list for all the playlists
df = pd.DataFrame()

# looping through all the playlists
for playlist_url in tqdm(playlists["playlist_url"]):
    # getting the tracks
    track_list = get_playlist_tracks(playlist_url, access_token)

    # saving tracks
    df = pd.concat([df, track_list])

  0%|          | 0/4 [00:00<?, ?it/s]

In [36]:
print(df.shape)
df.head()

(435, 21)


,preview_url,available_markets,explicit,type,episode,track,album,artists,disc_number,track_number,...,external_ids,external_urls,href,id,name,popularity,uri,is_local,playlist_url,playlist_id
0,https://p.scdn.co/mp3-preview/5f331286613eba71...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'FR9Z62100812'},{'spotify': 'https://open.spotify.com/track/0x...,https://api.spotify.com/v1/tracks/0x29CeLQLR31...,0x29CeLQLR31LY4OtWkcPp,Floating on Silence,48,spotify:track:0x29CeLQLR31LY4OtWkcPp,False,https://open.spotify.com/playlist/37i9dQZF1DXb...,37i9dQZF1DXbXD9pMSZomS
1,https://p.scdn.co/mp3-preview/b0eb532193111646...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'SE5Q52203695'},{'spotify': 'https://open.spotify.com/track/5d...,https://api.spotify.com/v1/tracks/5dCkryH3vx41...,5dCkryH3vx41J2FDsbmxqq,The Hills,46,spotify:track:5dCkryH3vx41J2FDsbmxqq,False,https://open.spotify.com/playlist/37i9dQZF1DXb...,37i9dQZF1DXbXD9pMSZomS
2,https://p.scdn.co/mp3-preview/2ec7f3fa967937a1...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'GBCFB2000760'},{'spotify': 'https://open.spotify.com/track/69...,https://api.spotify.com/v1/tracks/69uPNh3b6VKd...,69uPNh3b6VKdMZMbIKYQ1l,By Your Side,45,spotify:track:69uPNh3b6VKdMZMbIKYQ1l,False,https://open.spotify.com/playlist/37i9dQZF1DXb...,37i9dQZF1DXbXD9pMSZomS
3,https://p.scdn.co/mp3-preview/2184855d4a8ff5fa...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'GBCEL2400346'},{'spotify': 'https://open.spotify.com/track/23...,https://api.spotify.com/v1/tracks/23glDLdns3Jn...,23glDLdns3JnzpaFdY8eH8,RITUAL (palace),43,spotify:track:23glDLdns3JnzpaFdY8eH8,False,https://open.spotify.com/playlist/37i9dQZF1DXb...,37i9dQZF1DXbXD9pMSZomS
4,https://p.scdn.co/mp3-preview/ddbaa2baf5d69d28...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,{'isrc': 'FR9Z62000349'},{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51chgK66fLkr...,51chgK66fLkr1TO3zRYt79,Beaches,41,spotify:track:51chgK66fLkr1TO3zRYt79,False,https://open.spotify.com/playlist/37i9dQZF1DXb...,37i9dQZF1DXbXD9pMSZomS


In [37]:
# getting audio features for all the tracks in the playlist
af_df = get_audio_features(df["id"].values, access_token)
af_df

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature
0,0,0.716,0.6450,11,-5.894,0,0.0402,0.0459,0.8520,0.0691,0.960,123.009,0x29CeLQLR31LY4OtWkcPp,4
1,1,0.817,0.4780,6,-7.892,0,0.0707,0.0405,0.9160,0.1020,0.646,117.018,5dCkryH3vx41J2FDsbmxqq,4
2,2,0.887,0.8230,5,-7.570,0,0.0462,0.0237,0.7310,0.5930,0.641,121.993,69uPNh3b6VKdMZMbIKYQ1l,4
3,3,0.481,0.3890,11,-15.531,1,0.0282,0.3210,0.9540,0.0822,0.171,132.986,23glDLdns3JnzpaFdY8eH8,4
4,4,0.615,0.6480,2,-9.515,1,0.0472,0.1400,0.8670,0.1460,0.418,126.027,51chgK66fLkr1TO3zRYt79,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,430,0.708,0.1880,0,-19.226,1,0.0504,0.9070,0.9020,0.1060,0.469,76.522,098POWyt9OYxpTbzosh47I,4
431,431,0.571,0.4220,9,-12.911,0,0.0268,0.9230,0.9480,0.1150,0.516,83.204,61cdd5FDCdse6DYaN8ZMXe,4
432,432,0.584,0.1390,5,-11.483,1,0.0304,0.9610,0.0369,0.1060,0.240,106.759,2ghcpXt1t7oQfHcF5hUBP2,4
433,433,0.449,0.1420,10,-18.201,1,0.0421,0.9450,0.9140,0.1110,0.131,112.211,73LXSkZdpjWC4XUDTju8LK,4


In [38]:
# merging the two dataframes
final_df = pd.merge(df, af_df, on="id")
final_df.tail()

,preview_url,available_markets,explicit,type,episode,track,album,artists,disc_number,track_number,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
430,https://p.scdn.co/mp3-preview/578b656fa6a51603...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,2,...,0,-19.226,1,0.0504,0.907,0.9020,0.106,0.469,76.522,4
431,https://p.scdn.co/mp3-preview/13d359949bcd29ad...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,9,-12.911,0,0.0268,0.923,0.9480,0.115,0.516,83.204,4
432,https://p.scdn.co/mp3-preview/8a1385f197225d52...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,5,-11.483,1,0.0304,0.961,0.0369,0.106,0.240,106.759,4
433,https://p.scdn.co/mp3-preview/285fc1045ab80497...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,...,10,-18.201,1,0.0421,0.945,0.9140,0.111,0.131,112.211,4
434,None,"[AD, AE, AR, AT, AU, BD, BE, BG, BH, BO, BR, C...",False,track,False,True,"{'available_markets': ['AD', 'AE', 'AR', 'AT',...",[{'external_urls': {'spotify': 'https://open.s...,1,8,...,5,-17.157,0,0.0459,0.995,0.7590,0.105,0.642,69.861,3


In [39]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   preview_url        378 non-null    object 
 1   available_markets  435 non-null    object 
 2   explicit           435 non-null    bool   
 3   type               435 non-null    object 
 4   episode            435 non-null    bool   
 5   track              435 non-null    bool   
 6   album              435 non-null    object 
 7   artists            435 non-null    object 
 8   disc_number        435 non-null    int64  
 9   track_number       435 non-null    int64  
 10  duration_ms        435 non-null    int64  
 11  external_ids       435 non-null    object 
 12  external_urls      435 non-null    object 
 13  href               435 non-null    object 
 14  id                 435 non-null    object 
 15  name               435 non-null    object 
 16  popularity         435 non

In [40]:
# saving to csv
final_df.to_csv("./data/playlist_tracks.csv", index=False)
